## Description

 - The code is adapted from a task to find the most conserved regions of 100-200 bp in 10 SARS-Cov-2 genomes without performing MSA.
 - A total of eleven SARS-Cov-2 genomes were downloaded from NCBI [https://www.ncbi.nlm.nih.gov/sars-cov-2/] 
 
## Result & Discussion

 - A total of 184 conserved regions of 100 bp were mapped in the 10 genomes. Because the eD value is 0, these conserved regions are in fact identical in all genomes.
 - This result is expected due to the fact that the genomes we analyzed are simply potential different variants of the same virus.

In [313]:
import numpy as np
import pandas as pd

class genome(object):
    """read a .fasta and assign the sequence to a variable"""
    def __init__(self, filename):
        self.filename = filename
        
    def readgenome(self, filename):
        gnome = ''
        with open(filename, 'r') as f:
            for line in f:
                if not line[0] == '>':
                    gnome += line.rstrip()
        return gnome
    

def jjGlobal_editDist(x, y):
    """find the best alignment between x and y and return edit distance value and alignment position in y"""
    D = []
    for i in range(len(x)+1):
        D.append([0]* (len(y)+1)) 
        #create a matrix of all zeros that has (len(x)+1) rows and (len(y)+1) columns
    
    for i in range(1, len(x)+1):
        D[i][0] = D[i-1][0] + 1 #first column with numerically increase numbers from (0, len(x)+1)
    for i in range(1, len(y)+1):
        D[0][i] = D[0][i-1] # the fist row remain zero values
    
    count = 0
    for i in range(1, len(x)+1):
        for j in range(1, len(y)+1):
            distHor = D[i][j-1] + 1
            distVer = D[i-1][j] + 1
            if x[i-1] == y[j-1]:
                distDiag = D[i-1][j-1]
            else: 
                distDiag = D[i-1][j-1] + 1
            
            D[i][j] = min(distHor, distVer, distDiag)
            #count += 1
        
        D_pos = j - i
            
    return min(D[len(x)]), D_pos


class g_eD(object):
    """generate a list for each genome (g) using the queries and Oleg_editDist function"""
    def __init__(self, g):
        self.g = g

    def eD(self, g):
        self.g = g
        eD = []
        for (seg, start) in ref_seg_id:
            eD.append(Oleg_editDist(seg, g))
        return eD
    
class eD_item (object):
    """for each eD, generate two lists containing the eD value and pos info, respectively"""
    def __init__(self, eD):
        self.eD = eD
        
    def eD_val(self, eD):
        self.eD = eD
        eD_val_list = []
        eD_val_list.append([val for (val, pos) in eD])
        return eD_val_list
    
    def eD_pos(self, eD):
        self.eD = eD
        eD_pos_list = []
        eD_pos_list.append(pos for (val,pos) in eD)
        return eD_pos_list

In [314]:
# create a list containing the 10 genome sequences: gnome_list

g_name = ['MZ401444.fasta', 'MZ401459.fasta', 'MZ401492.fasta', 'MZ401506.fasta', 'MZ401850.fasta', 'MZ402172.fasta', 'MZ402197.fasta', 'MZ402259.fasta', 'MZ402465.fasta', 'MZ402490.fasta']
gnome_list = []
for name in g_name:
    gn = genome(name)
    gnome_list.append(gn.readgenome(gn.filename))
    
# unpack gnome_list so that each genome can be assigned to variables from g1 to g10 in the new list: g_list
g1, g2, g3, g4, g5, g6, g7, g8, g9, g10 = gnome_list
g_list = [g1, g2, g3, g4, g5, g6, g7, g8, g9, g10]

In [315]:
# obtain reference genome to be used for generation of queries

ref = genome('NC045512_Ref.fasta')
g_ref = ref.readgenome(ref.filename)

def segment_id(genome, L):
    """generates a list of tuples (seq, pos) containing all segments (length=L) after slicing the genome """
    segment_id = []
    sg_seq = ''
    sg_start= ''
    count = 0
    for i in range(len(genome)//L):
        sg_seq = genome[i*L:(i*L+L)]
        count +=1
        sg_start = i*L
        segment_id.append((sg_seq,sg_start))  
    
    return segment_id

ref_seg_id = segment_id(g_ref, 100) #list of query sequences with position info generated from the reference genome

In [316]:
# create a list of eD_list, and unpack so that eD1 = eD(g1), ...
# eD1 is a list of tuples from g1, each tuple (minD, [D_loc]) corresponds to a query

eD_list = []
for g in g_list:
    gEd = g_eD(g)
    eD_list.append(gEd.eD(gEd.g))
    
eD1, eD2, eD3, eD4, eD5, eD6, eD7, eD8, eD9, eD10 = eD_list
g_eD_list = [eD1, eD2, eD3, eD4, eD5, eD6, eD7, eD8, eD9, eD10]

In [317]:
# generate the eD value and pos info lists for all genomes in two separate lists

val_list = []
pos_list = []
for eD in g_eD_list:
    gItem = eD_item(eD)
    val_list.append(gItem.eD_val(gItem.eD))
    pos_list.append(gItem.eD_pos(gItem.eD)[0])

In [318]:
#combine the lists of eD values and pos from the 10 genomes into arrays, respectively

val_matrix = np.array(val_list, dtype=object)
pos_matrix = np.array(pos_list, dtype=object)

eD_sum = np.sum(val_matrix, axis=0) # return 1D array containing the sum of eD values for each query
eD_min = np.amin(eD_sum) # find the lowest eD value in the array
eD_min_index = np.where(eD_sum==eD_min) # Extract indices where the lowest eD value was observed for all genomes

# 'eD_min_index' is a tuple containing two arrays, needs to extract the 2nd array, and converted it to a list
(_,eD_index)= eD_min_index #extract the 2nd array in the tuple
eD_index = eD_index.tolist() #change array to list

In [319]:
# make df of the extracted array and use df._csv to save it as a .csv file (output)
df = pd.DataFrame(con_region)
df.to_csv('con_reg.csv') # the file is renamed to con_region.csv for submission

# obtain the list of conserved sequences with numerical labels with was included in con_region.csv
ref_seg = [seg for (seg, ID) in ref_seg_id]
r_seg = np.array(ref_seg)
c_seg = list(r_seg[eD_index])
df = pd.DataFrame(c_seg)
df.to_csv('con_regSeq.csv')